In [ ]:
%load_ext autoreload
%autoreload 2

# Dataset Generation

In [1]:
# dataset builder
from jenny.dataset_generation import BatchDatasetGenerator

# feature generators
from jenny.dataset_generation import (
    IdFeatureGenerator,
    TimestampFeatureGenerator,
    NumericFeatureGenerator,
    TimeSensitiveNumericFeatureGenerator,
    TextFeatureGenerator,
    CategoryFeatureGenerator
)

# time series modeling
from jenny.dataset_generation import (
    Trend, Seasonality, Noise, NormalPercentageDeviation
)

import pandas as pd

In [2]:
# dataset spec
dataset_generator = BatchDatasetGenerator(
    events_generator=TimeSensitiveNumericFeatureGenerator(
        name="events",
        trend=Trend(
            base_value=100,
            slope=1,
        ),
        seasonality=Seasonality(
            week_days=[0.85, 0.9, 1, 1, 0.9, 0.8, 0.8],
            month_period=[1, 0.9, 1],
            year_months=[1, 0.95, 0.9, 0.9, 0.85, 0.85, 0.85, 0.85, 0.9, 0.9, 0.95, 1],
        ),
        noise=Noise(
            var=0.05,
            seed=123,
        )
    ),
    features=[
        IdFeatureGenerator(
            name="id",
            min_id=1000,
            monotonically_increase=True
        ),
        TimestampFeatureGenerator(
            name="ts"
        ),
        NumericFeatureGenerator(
            name="numeric_feature",
            base_value=500,
            percentage_deviation_generator=NormalPercentageDeviation(
                var=0.8,
                seed=123,
            )
        ),
        TimeSensitiveNumericFeatureGenerator(
            name="numeric_feature_with_trend",
            trend=Trend(
                base_value=500,
                slope=10,
            ),
            noise=Noise(
                var=0.05,
                seed=123,
            )
        ),
        TimeSensitiveNumericFeatureGenerator(
            name="numeric_feature_with_seasonality",
            trend=Trend(
                base_value=500,
                slope=10,
            ),
            noise=Noise(
                var=0.05,
                seed=123,
            ),
            seasonality=Seasonality(
                week_days=[0.9, 1.1, 1.1, 1, 0.9, 0.9, 0.9],
                year_months=[1.1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.2],
            )
        ),
        TextFeatureGenerator(
            name="description",
            max_base_length=120,
        ),
        CategoryFeatureGenerator(
            name="category",
            categories=["A", "B", "C", "D", "E"]
        )
    ]
)

In [3]:
from datetime import datetime, timezone

start_ts = datetime(year=2018, month=1, day=1, tzinfo=timezone.utc)
dataset_pdf = dataset_generator.generate(start_ts=start_ts, n=365*2)

In [4]:
# showing generated df
dataset_pdf

,id,ts,numeric_feature,numeric_feature_with_trend,numeric_feature_with_seasonality,description,category
0,1000,2018-01-01 23:49:52+00:00,104.351460,475.271966,470.519247,Born fish teach foot beautiful specific federa...,B
1,1001,2018-01-01 07:24:50+00:00,352.885339,490.805334,485.897280,Prepare service girl approach hold across moth...,D
2,1002,2018-01-01 03:13:05+00:00,1015.170105,532.198132,526.876150,Practice want girl report loss front. Around d...,B
3,1003,2018-01-01 11:34:57+00:00,577.589768,504.849360,499.800867,Democrat several board itself ready film civil...,C
4,1004,2018-01-01 08:10:51+00:00,868.092360,523.005772,517.775715,Half business baby join. Third research a patt...,B
...,...,...,...,...,...,...,...
772,266160,2019-12-31 20:08:21+00:00,148.068049,7447.306262,9830.444266,Half environment visit need career believe vot...,E
773,266161,2019-12-31 08:35:17+00:00,549.178644,7837.887705,10346.011770,Leader spend state hand about add leader. Prof...,A
774,266162,2019-12-31 16:45:00+00:00,1115.078481,8388.932671,11073.391125,Only run respond medical wife sing. Really env...,E
775,266163,2019-12-31 16:50:42+00:00,244.543546,7541.249278,9954.449046,Professor Mr check owner television. Something...,B


In [5]:
dataset_pdf.dtypes

id                                                int64
ts                                  datetime64[ns, UTC]
numeric_feature                                 float64
numeric_feature_with_trend                      float64
numeric_feature_with_seasonality                float64
description                                      string
category                                         string
dtype: object

# Profiling Pipeline

In [6]:
# create spark session and convert to spark df
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('jenny').getOrCreate()
spark.conf.set("spark.sql.session.timeZone", "UTC")
dataset_df = spark.createDataFrame(dataset_pdf)

21/07/23 01:15:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
# profiling spec

from jenny.profiling.profilers import (
    GlobalProfiler,
    ColumnStatsProfiler,
    CompletenessProfiler,
)
from jenny.profiling.pipeline import ProfilingPipeline

metrics = ["max", "min", "sum", "avg"]
profiling_pipeline = ProfilingPipeline(
    profilers=[
        GlobalProfiler(),
#         ColumnStatsProfiler(
#             from_column="id",
#             metrics=metrics
#         ),
#         ColumnStatsProfiler(
#             from_column="numeric_feature",
#             metrics=metrics
#         ),
#         CompletenessProfiler(
#             from_column="numeric_feature",
#         ),
#         ColumnStatsProfiler(
#             from_column="numeric_feature_with_trend",
#             metrics=metrics
#         ),
#         ColumnStatsProfiler(
#             from_column="numeric_feature_with_seasonality",
#             metrics=metrics
#         ),
#         ColumnStatsProfiler(
#             from_column="description",
#             metrics=metrics,
#             not_numeric=True
#         ),
#         ColumnStatsProfiler(
#             from_column="category",
#             metrics=metrics,
#             not_numeric=True
#         ),
#         ColumnStatsProfiler(
#             from_column="category",
#             metrics=["count_distinct"]
#         )
    ]
)

In [8]:
# run profiling 
profiling_df = profiling_pipeline.run(
    input_df=dataset_df,
    time_column="ts"
)

In [9]:
profiling_df.sort("ts").toPandas()

21/07/23 01:15:36 WARN TaskSetManager: Stage 0 contains a task of very large size (2786 KiB). The maximum recommended task size is 1000 KiB.


,ts,global__count
0,2018-01-01,80
1,2018-01-02,89
2,2018-01-03,108
3,2018-01-04,103
4,2018-01-05,97
...,...,...
725,2019-12-27,667
726,2019-12-28,683
727,2019-12-29,633
728,2019-12-30,756


In [10]:
profiling_df.sort("ts").show()

21/07/23 01:15:40 WARN TaskSetManager: Stage 5 contains a task of very large size (2786 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+-------------+
|                 ts|global__count|
+-------------------+-------------+
|2018-01-01 00:00:00|           80|
|2018-01-02 00:00:00|           89|
|2018-01-03 00:00:00|          108|
|2018-01-04 00:00:00|          103|
|2018-01-05 00:00:00|           97|
|2018-01-06 00:00:00|           86|
|2018-01-07 00:00:00|           82|
|2018-01-08 00:00:00|           93|
|2018-01-09 00:00:00|           95|
|2018-01-10 00:00:00|          107|
|2018-01-11 00:00:00|           99|
|2018-01-12 00:00:00|           83|
|2018-01-13 00:00:00|           85|
|2018-01-14 00:00:00|           78|
|2018-01-15 00:00:00|           91|
|2018-01-16 00:00:00|           93|
|2018-01-17 00:00:00|          112|
|2018-01-18 00:00:00|          101|
|2018-01-19 00:00:00|           94|
|2018-01-20 00:00:00|           87|
+-------------------+-------------+
only showing top 20 rows



# Time Series Forecast Models Cross-Validation

![Day Forward-Chaining](https://miro.medium.com/max/4800/1*gYTT2d-Suszciijr10l7iQ.png)

In [22]:
from jenny.anomaly.evaluation import CrossValidation, ModelValidationSpec
cross_validation = CrossValidation(
    validation_specs={
        ModelValidationSpec(name="prophet", model_kwargs_list=[
            {"seasonality_mode": "multiplicative"},
            {"seasonality_mode": "additive"}
        ]),
        ModelValidationSpec(name="neural-prophet", model_kwargs_list=[
            {"seasonality_mode": "multiplicative"},
            {"seasonality_mode": "additive"}
        ])
    },
    ts_column="ts",
    feature_column="global__count",
    start_proportion=0.75,
    slide=8
)

In [23]:
cross_validation_metrics_results = cross_validation.run(data_pdf=profiling_df.sort("ts").toPandas())

21/07/23 01:49:24 WARN TaskSetManager: Stage 17 contains a task of very large size (2786 KiB). The maximum recommended task size is 1000 KiB.
INFO: nprophet.utils - set_auto_seasonalities: Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO:nprophet.utils:Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO: nprophet.utils - set_auto_seasonalities: Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO:nprophet.utils:Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 8
INFO:nprophet.config:Auto-set batch_size to 8
INFO: nprophet.config - set_auto_batch_epoch: Auto-set epochs to 58
INFO:nprophet.config:Auto-set epochs to 58


/Users/temp/.pyenv/versions/3.9.6/envs/jenny-3.9.6/lib/python3.9/site-packages/torch/nn/modules/container.py:552: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")
INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 6.58E-03
INFO:nprophet:learning rate range test found optimal lr: 6.58E-03
Epoch[58/58]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:05<00:00, 10.72it/s, SmoothL1Loss=0.00137, MAE=19.2, RegLoss=0]
/Users/temp/.pyenv/versions/3.9.6/envs/jenny-3.9.6/lib/python3.9/site-packages/torch/nn/modules/container.py:552: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")
INFO: nprophet.utils - set_auto_seasonalities: Disabling yearly seasonality. Run NeuralProphet with yearly_s

/Users/temp/.pyenv/versions/3.9.6/envs/jenny-3.9.6/lib/python3.9/site-packages/torch/nn/modules/container.py:552: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")
INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 5.34E-01
INFO:nprophet:learning rate range test found optimal lr: 5.34E-01
Epoch[58/58]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:05<00:00, 10.96it/s, SmoothL1Loss=0.00142, MAE=19.6, RegLoss=0]
/Users/temp/.pyenv/versions/3.9.6/envs/jenny-3.9.6/lib/python3.9/site-packages/torch/nn/modules/container.py:552: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")
INFO: nprophet.utils - set_auto_seasonalities: Disabling yearly seasonality. Run NeuralProphet with yearly_s

/Users/temp/.pyenv/versions/3.9.6/envs/jenny-3.9.6/lib/python3.9/site-packages/torch/nn/modules/container.py:552: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")
INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 6.58E-01
INFO:nprophet:learning rate range test found optimal lr: 6.58E-01
Epoch[58/58]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:05<00:00, 11.23it/s, SmoothL1Loss=0.00142, MAE=19.6, RegLoss=0]
/Users/temp/.pyenv/versions/3.9.6/envs/jenny-3.9.6/lib/python3.9/site-packages/torch/nn/modules/container.py:552: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")
INFO: nprophet.utils - set_auto_seasonalities: Disabling yearly seasonality. Run NeuralProphet with yearly_s

/Users/temp/.pyenv/versions/3.9.6/envs/jenny-3.9.6/lib/python3.9/site-packages/torch/nn/modules/container.py:552: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")
INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 6.58E-01
INFO:nprophet:learning rate range test found optimal lr: 6.58E-01
Epoch[57/57]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:05<00:00, 11.10it/s, SmoothL1Loss=0.00128, MAE=17.7, RegLoss=0]
/Users/temp/.pyenv/versions/3.9.6/envs/jenny-3.9.6/lib/python3.9/site-packages/torch/nn/modules/container.py:552: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")
INFO: nprophet.utils - set_auto_seasonalities: Disabling yearly seasonality. Run NeuralProphet with yearly_s

...

Initial log joint probability = -4.13525
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1987.43    0.00944617       126.759           1           1      135   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     123       1988.66   0.000160702       115.739   2.311e-06       0.001      226  LS failed, Hessian reset 
     177       1989.39   0.000113388       79.9144   6.846e-07       0.001      379  LS failed, Hessian reset 
     199       1989.52   0.000214635       66.5535           1           1      408   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     249       1989.68   9.02079e-05        64.507   1.309e-06       0.001      508  LS failed, Hessian reset 
     299       1989.69   5.96309e-05       72.6515           1           1      577   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Note

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -4.13525
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      85       1930.43   0.000740059       161.944   8.195e-06       0.001      154  LS failed, Hessian reset 
      99       1931.12     0.0037998        134.96      0.5478           1      178   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     137       1931.81   0.000196584       135.918   1.825e-06       0.001      270  LS failed, Hessian reset 
     199        1932.1   0.000789147       117.862           1           1      344   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     217       1932.29   0.000207903       144.945   2.336e-06       0.001      414  LS failed, Hessian reset 
     299       1932.52    0.00131297       97.4866           1           1      523   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Note

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -4.11408
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1990.62   0.000610052       172.981      0.6798      0.6798      143   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     191       1992.47   0.000158189       120.904   1.698e-06       0.001      296  LS failed, Hessian reset 
     199       1992.58    0.00147884       190.726           1           1      308   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     280       1992.83    2.8341e-06       75.8328   4.159e-08       0.001      458  LS failed, Hessian reset 
     296       1992.83   1.06378e-07        62.753     0.01858           1      482   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [26]:
for metrics in cross_validation_metrics_results:
    print(metrics.name, f"avg error={metrics.errors_avg}, min_error={metrics.errors_min}, max error={metrics.errors_max}, std error={metrics.errors_stddev}\n")

neural-prophet avg error=0.11807321550842209, min_error=0.00546133281379346, max error=1.0, std error=0.19233509080453068

prophet avg error=0.06918603569797034, min_error=0.006327105257649969, max error=0.14285352393165335, std error=0.0425779548913361

